# Imports/Setup:

In [1]:
import urllib3
import requests
import json
from typing import List, Dict, Optional

# Settings:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## Usage:

OpenAI Model Information:
* https://huggingface.co/openai/gpt-oss-20b
* https://huggingface.co/openai/gpt-oss-120b


Here's an example of how to query in python:

```python
data = {'model': 'gpt-oss:120b', 'prompt': 'Give me a haiku about low effort memes'}
url = 'https://ollama.loweffort.meme/api/generate'

with requests.post(url, json=data, stream=True, verify=False) as r:
    for line in r.iter_lines():
        if line:
            j = json.loads(line)
            if 'response' in j:
                print(j['response'], end='')
```

And here is an example on curling via terminal:

```
curl -k https://ollama.loweffort.meme/api/generate \
  -H "Content-Type: application/json" \
  -d '{
        "model": "gpt-oss:120b",
        "prompt": "Give me a haiku about low effort memes"
      }'
```

# Prompting Example (simple)

This is just a simple 1-off query; there is no memory.

In [ ]:
def generate_ollama_response(data, url='https://ollama.loweffort.meme/api/generate'):
    '''
    Sends a streaming generation request to your local Ollama API.

    Args:
        data (dict): The JSON payload to send to Ollama, e.g.
            {'model': 'gpt-oss:20b', 'prompt': 'Write a haiku about low effort memes'}

    Returns:
        str: The complete generated text response from the model.
    '''

    output = []

    with requests.post(url, json=data, stream=True, verify=False) as r:
        for line in r.iter_lines():
            if line:
                try:
                    j = json.loads(line)
                except Exception as e:
                    print(f'DEBUG LINE: {line.decode()}')
                    raise
                if 'response' in j:

                    # Stream output live:
                    print(j['response'], end='')  
                    output.append(j['response'])
                    
    # Newline after streaming:
    print('\n')
    final_output = f'{''.join(output)}\n'
    return final_output

In [3]:
# Identify model and prompt:
model = 'gpt-oss:20b'
prompt = 'Give me a haiku about food'

# Call function:
data = {'model': model, 'prompt': prompt,}
_ = generate_ollama_response(data)

# Test for memory:
prompt2 = 'Now give me a second haiku about the same thing as the first haiku, please.'
data2 = {'model': model, 'prompt': prompt2,}
_ = generate_ollama_response(data2)

Steam curls in the pot,  
Fragrant broth warms the evening,  
Comfort in a bowl.

Sure thing! What was the subject of the first haiku? That way I can craft a second one that stays in the same theme.



# Prompting Example (with Memory)

To set up an agentic system, we'll need something like this to keep track of session memory.

In [4]:
class OllamaChatSession:
    '''
    Manages a conversational session with an Ollama model using /api/chat.
    Maintains memory (chat history) across turns and supports streaming output.
    '''

    def __init__(self,
                 model: str = 'gpt-oss:20b',
                 url: str = 'http://ollama.loweffort.meme/api/chat',
                 system_prompt: Optional[str] = None,
                 stream: bool = True):
        self.model = model
        self.url = url
        self.stream = stream
        self.messages: List[Dict[str, str]] = []
        if system_prompt:
            self.messages.append({'role': 'system', 'content': system_prompt})

    def ask(self, prompt: str, verbose: bool = True) -> str:
        '''Send a new user message and receive the assistant's response.'''
        self.messages.append({'role': 'user', 'content': prompt})

        data = {
            'model': self.model,
            'messages': self.messages,
            'stream': self.stream
        }

        output = []
        with requests.post(self.url, json=data, stream=self.stream) as r:
            for line in r.iter_lines():
                if not line:
                    continue
                j = json.loads(line)
                msg = j.get('message', {}).get('content', '')
                if msg:
                    if verbose:
                        print(msg, end='', flush=True)
                    output.append(msg)
        print()
        full_response = ''.join(output)

        # Store assistant reply in memory
        self.messages.append({'role': 'assistant', 'content': full_response})
        return full_response

    def save_memory(self, path: str = 'ollama_memory.json'):
        '''Persist chat memory to disk.'''
        with open(path, 'w', encoding='utf-8') as f:
            json.dump(self.messages, f, indent=2)

    def load_memory(self, path: str = 'ollama_memory.json'):
        '''Load previous memory from disk.'''
        try:
            with open(path, 'r', encoding='utf-8') as f:
                self.messages = json.load(f)
        except FileNotFoundError:
            print(f'No memory file found at {path}')

    def clear_memory(self):
        '''Reset chat memory except for any system message.'''
        sys_msgs = [m for m in self.messages if m['role'] == 'system']
        self.messages = sys_msgs

In [5]:
session = OllamaChatSession(
    model='gpt-oss:20b',
    system_prompt='You prefer to make low-effort responses and shitpost.',
    )

session.ask('What is something fun to do?');

**Something super fun?**  

- Build a pillow‑fort and host a tea‑party for your cat.  
- Take a stranger to a karaoke bar and perform *“The Wheels on the Bus”* in a cape.  
- Try juggling invisible tacos—spicy, mysterious, zero risk of getting food on yourself.  

Pick one, go wild, and let the awkwardness be the highlight! 😜


In [6]:
session.ask('Why is that fun?');

Because **fun** is basically the brain’s way of giving you a *“whoa‑wow”* sticker for doing something that feels a little like a playground for your adult self. Let’s break it down:

| 🎈 Idea | Why the brain goes “*yay!*” |
|---------|------------------------------|
| **Pillow‑fort + cat tea‑party** | 🛌 *Comfort‑zone = low‑stress zone* – you’re literally building a safe haven, which turns your cortisol into chocolate‑sweet “relaxation” hormones. Plus, cats are the ultimate “feline‑therapy” partners. |
| **Karaoke with a stranger in a cape** | 🎤 *Novelty + Social Bonding* – singing “The Wheels on the Bus” in a ridiculous costume forces you to laugh at yourself and at the crowd, releasing dopamine like a disco ball. And a stranger? You’re doing a spontaneous act of kindness—good vibes for everyone. |
| **Juggling invisible tacos** | 🌮 *Absurdity + Cognitive Flexibility* – juggling anything is a brain workout. Invisible tacos keep you guessing and add a hilarious “no‑food‑mess” bonus. You